## [Start Milvus](https://milvus.io/docs/install_standalone-docker.md#Start-Milvus)

```bash
sudo docker compose run -d
```

- Check everything runs correctly

In [1]:
!docker compose ps


NAME                IMAGE                                      COMMAND                  SERVICE             CREATED             STATUS                            PORTS
milvus-etcd         quay.io/coreos/etcd:v3.5.5                 "etcd -advertise-cli…"   etcd                6 minutes ago       Up 3 minutes (healthy)            2379-2380/tcp
milvus-minio        minio/minio:RELEASE.2023-03-20T20-16-18Z   "/usr/bin/docker-ent…"   minio               6 minutes ago       Up 4 seconds (health: starting)   0.0.0.0:9000-9001->9000-9001/tcp, :::9000-9001->9000-9001/tcp
milvus-standalone   milvusdb/milvus:v2.3.3                     "/tini -- milvus run…"   standalone          6 minutes ago       Up 3 seconds (health: starting)   0.0.0.0:9091->9091/tcp, :::9091->9091/tcp, 0.0.0.0:19530->19530/tcp, :::19530->19530/tcp


- Get the port of Milvus:

In [2]:
!docker port milvus-standalone 19530/tcp


0.0.0.0:19530
[::]:19530


## Example Run

- Imports a PyMilvus package:


In [3]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

- Connects to a server:


In [4]:
connections.connect("default", host="localhost", port="19530")


- Creates a collection:
 

In [5]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)
hello_milvus

<Collection>:
-------------
<name>: hello_milvus
<description>: hello_milvus is the simplest demo to introduce the APIs
<schema>: {'auto_id': False, 'description': 'hello_milvus is the simplest demo to introduce the APIs', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'random', 'description': '', 'type': <DataType.DOUBLE: 11>}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 8}}]}

- Inserts vectors in the collection:


In [6]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)
hello_milvus.flush()  


- Builds indexes on the entities:


In [7]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)


Status(code=0, message=)

- Loads the collection to memory and performs a vector similarity search:


In [8]:
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])
result

['["id: 2998, distance: 0.0, entity: {\'random\': -19.0}", "id: 1964, distance: 0.14202311635017395, entity: {\'random\': -19.0}", "id: 954, distance: 0.15727858245372772, entity: {\'random\': -12.0}"]', '["id: 2999, distance: 0.0, entity: {\'random\': -16.0}", "id: 2824, distance: 0.07784067839384079, entity: {\'random\': -14.0}", "id: 2308, distance: 0.08826693892478943, entity: {\'random\': -20.0}"]']

- Performs a vector query:


In [9]:
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"])
result

[{'random': -11.0,
  'embeddings': [0.4310956,
   0.80701256,
   0.006272254,
   0.32876307,
   0.47871944,
   0.1541556,
   0.7061802,
   0.29775092],
  'pk': 0},
 {'random': -11.0,
  'embeddings': [0.9891824,
   0.21191564,
   0.18519707,
   0.5150672,
   0.7217334,
   0.8299412,
   0.85603243,
   0.9334412],
  'pk': 1},
 {'random': -11.0,
  'embeddings': [0.278978,
   0.47840562,
   0.24804264,
   0.80123615,
   0.0477932,
   0.30448806,
   0.10127121,
   0.6240707],
  'pk': 10},
 {'random': -13.0,
  'embeddings': [0.17773424,
   0.16972134,
   0.2824241,
   0.43726888,
   0.79575914,
   0.28628102,
   0.031056263,
   0.32426533],
  'pk': 11},
 {'random': -12.0,
  'embeddings': [0.32377836,
   0.18128197,
   0.44111878,
   0.117649354,
   0.9091777,
   0.43493578,
   0.9703712,
   0.55288714],
  'pk': 15},
 {'random': -13.0,
  'embeddings': [0.9677489,
   0.6271182,
   0.61705256,
   0.50528806,
   0.9603657,
   0.44799015,
   0.295471,
   0.15103668],
  'pk': 20},
 {'random': -12.0

- Performs a hybrid search:


In [10]:
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > -12", output_fields=["random"])
result

['["id: 2010, distance: 0.2405138909816742, entity: {\'random\': -11.0}", "id: 781, distance: 0.24054594337940216, entity: {\'random\': -11.0}", "id: 10, distance: 0.25380197167396545, entity: {\'random\': -11.0}"]', '["id: 1143, distance: 0.1580730378627777, entity: {\'random\': -11.0}", "id: 647, distance: 0.20683568716049194, entity: {\'random\': -11.0}", "id: 2188, distance: 0.33659517765045166, entity: {\'random\': -11.0}"]']

- Deletes entities by their primary keys:


In [11]:
expr = f"pk in [{entities[0][0]}, {entities[0][1]}]"
hello_milvus.delete(expr)


(insert count: 0, delete count: 2, upsert count: 0, timestamp: 445593116497936387, success count: 0, err count: 0)

- Drops the collection:


In [12]:
utility.drop_collection("hello_milvus")


Congratulations! You have started Milvus standalone and performed your first vector similarity search.

## [Stop Milvus](https://milvus.io/docs/install_standalone-docker.md#Stop-Milvus)

In [13]:
!docker compose down

[+] Running 0/0
 ⠋ Container milvus-standalone  Stopping                                   0.1s 
[+] Running 0/1
 ⠙ Container milvus-standalone  Stopping                                   0.2s 
[+] Running 0/1
 ⠹ Container milvus-standalone  Stopping                                   0.3s 
[+] Running 0/1
 ⠸ Container milvus-standalone  Stopping                                   0.4s 
[+] Running 0/1
 ⠼ Container milvus-standalone  Stopping                                   0.5s 
[+] Running 0/1
 ⠴ Container milvus-standalone  Stopping                                   0.6s 
[+] Running 0/1
 ⠦ Container milvus-standalone  Stopping                                   0.7s 
[+] Running 0/1
 ⠧ Container milvus-standalone  Stopping                                   0.8s 
[+] Running 0/1
 ⠇ Container milvus-standalone  Stopping                                   0.9s 
[+] Running 0/1
 ⠏ Container milvus-standalone  Stopping                                   1.0s 
[+] Running 0/1
 ⠋ Container m

- To delete the data, delete the `volume` directory.